<a href="https://colab.research.google.com/github/faisalDnay/SmartApp/blob/master/Taalgeneratie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opdracht 2.1: Taalgeneratie
We gaan aan de slag met een Large Language Model van OpenAI van vóór ChatGPT: GPT2.0 (nou ja, een [Nederlandse afgeleide](https://huggingface.co/GroNLP/gpt2-small-dutch) dan).

- Het voordeel aan dit model is dat het klein genoeg is dat het overal wel draait.
- Het nadeel is dat het nogal slecht is. (OpenAI bracht niet voor niets pas de opvolger GPT3.0 commercieel uit).
- Toch bestaat het model uit 129.000.000 parameters, deze parameters staan vast.

Daarom gaan we met andere eigenschappen van het model aan de slag:

- We beginnen natuurlijk met de tekstuele input, oftewel de prompt.
- Daarna kijken we naar het aantal woorddelen (tokens in het Engels) dat de LLM terug mag geven.
- Daarna kijken we naar de temperatuur van het model. Dit is gerelateerd aan de waarschijnlijkheid dat het model andere woorden selecteert dan het best voorspelde woorddeel/token.
- Daarna kijken we naar het aantal alternatieve woorddelen dat het model overweegt.
- En uiteindelijk kijken we naar de straf dat het model geeft voor het herhalen van woorddelen.

**Het doel van deze opdracht is om de eigenschappen van het model zo in te stellen dat deze het antwoord geven dat we zoeken.**

## 1. Installatie
Weer hebben we als eerst een aantal modules nodig die geïnstalleerd moeten worden.

In [1]:
# @title 1.1. Installeer modules
!pip install -U bitsandbytes transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0



## 2. Modules importeren
Ook nu moeten er wat modules geïmporteerd worden.

In [2]:
# @title 2.1 Importeer modules
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import torch.nn.functional as F


## 3. Functie definieren
En we moeten dit keer één functie definieren, namelijk de functie die de LLM uitvoert. Ook nu is het niet nodig om deze code te begrijpen voor de oefening. Je mag het natuurlijk wel bekijken en proberen te begrijpen.

In [3]:
# @title 3.1 Definieer LLM-functie

def run_llm(prompt, max_new_tokens, temperature, top_k, repetition_penalty):
    model_name = "GroNLP/gpt2-small-dutch"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name
    )

    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        output_scores=True,
        return_dict_in_generate=True,
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_tokens = outputs.sequences[0]
    scores = outputs.scores

    print(f"Prompt: {prompt}")
    generated_text = tokenizer.decode(generated_tokens[len(inputs.input_ids[0]):], skip_special_tokens=True)
    print("Answer:", generated_text)
    print()

    for i, logits in enumerate(scores):
        probs = F.softmax(logits, dim=-1)

        topk_probs, topk_ids = torch.topk(probs, top_k, dim=-1)
        topk_probs = topk_probs[0].tolist()
        topk_ids = topk_ids[0].tolist()

        token_id = generated_tokens[len(inputs.input_ids[0]) + i].item()
        token_str = tokenizer.decode([token_id])
        token_prob = probs[0, token_id].item()

        print(f"Step {i+1}: Generated token: {token_str!r} (prob={token_prob:.4f})")
        print(f"Top {top_k} candidates:")
        for rank, (tok_id, prob) in enumerate(zip(topk_ids, topk_probs), start=1):
            tok_str = tokenizer.decode([tok_id])
            marker = "<-- chosen" if tok_id == token_id else ""
            print(f"  {rank}. Token: {tok_str!r}, Probability: {prob:.4f} {marker}")
        print()

# 4. Instellen en uitvoeren van het model

Hieronder zijn de eigenschappen van het model alvast ingesteld. De prompt van het model is "Wat is de hoofdstad van Nederland?"

Het doel van deze opdracht is om de eigenschappen zo in te stellen dat we het antwoord krijgen "De hoofdstad van Nederland is Amsterdam."

Speel met de eigenschappen, en probeer te achterhalen wat ze precies voor invloed hebben op het antwoord.

Probeer, wanneer je de juiste eigenschappen hebt gevonden, te begrijpen waarom dit inderdaad de juiste eigenschappen zijn en waarom het model doet deze doet.

Tips:
- max_new_tokens bepaalt hoeveel tokens het model terug mag geven. Hoeveel tokens heb je nodig?
- temperature bepaalt hoe waarschijnlijk het is dat het model een ander token selecteert dan het best voorspelde token. Een hogere temperature geeft dus meer willekeur en daarmee creativiteit aan het antwoord (tussen 0 en 1, exclusief 0).
- top_tokens geeft aan hoeveel tokens het model overweegt om het volgende token uit te kiezen.
- repetition_penalty bepaalt de straf die het model geeft aan het herhalen van tokens. Een hogere penalty zorgt dus voor minder herhaling (tussen 0 en 1, exclusief 0).

In [13]:
prompt = "Wat is de hoofdstad van Nederland?\n"
max_new_tokens = 7
temperature = 0.3
top_tokens = 3
repetition_penalty = 1.0

run_llm(prompt, max_new_tokens, temperature, top_tokens, repetition_penalty)

Prompt: Wat is de hoofdstad van Nederland?

Answer: De hoofdstad van Nederland is Amsterdam.

Step 1: Generated token: 'De' (prob=0.7724)
Top 3 candidates:
  1. Token: 'De', Probability: 0.7724 <-- chosen
  2. Token: "'", Probability: 0.1385 
  3. Token: '"', Probability: 0.0891 

Step 2: Generated token: ' hoofdstad' (prob=0.9980)
Top 3 candidates:
  1. Token: ' hoofdstad', Probability: 0.9980 <-- chosen
  2. Token: ' stad', Probability: 0.0020 
  3. Token: ' Nederlandse', Probability: 0.0000 

Step 3: Generated token: ' van' (prob=0.9960)
Top 3 candidates:
  1. Token: ' van', Probability: 0.9960 <-- chosen
  2. Token: ' is', Probability: 0.0040 
  3. Token: ',', Probability: 0.0000 

Step 4: Generated token: ' Nederland' (prob=0.9999)
Top 3 candidates:
  1. Token: ' Nederland', Probability: 0.9999 <-- chosen
  2. Token: ' de', Probability: 0.0001 
  3. Token: ' het', Probability: 0.0000 

Step 5: Generated token: ' is' (prob=0.9993)
Top 3 candidates:
  1. Token: ' is', Probability: 0